# ML Course 3: Classification:  Week 1 Assignment 1
## Predicting sentiment from product reviews
## GraphLab

The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started! see also https://github.com/smarthi/UnivOfWashington-Machine-Learning/blob/master/Classification/week1/module-2-linear-classifier-assignment-blank.ipynb
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
import graphlab
import math
import string

# Data preperation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = graphlab.SFrame('../../course-1/Week3/amazon_baby.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to vinorda@gmail.com and will expire on June 04, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1473720799.log


Now, let us see a preview of what the dataset looks like.

In [3]:
products.head(5)

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_puctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_puctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [6]:
# lets look at last item in list - last review
print review_without_puctuation[ len(review_without_puctuation) -1 ]

# and associated word_count feature for last sample point
print products[-1]['word_count']

I love this product very mush  I have bought many car shades earlier but they never protected me from direct sun exposure  But these really work well and It also Keeps the temperature of the car
{'and': 1, 'love': 1, 'shades': 1, 'it': 1, 'have': 1, 'really': 1, 'from': 1, 'temperature': 1, 'sun': 1, 'also': 1, 'i': 2, 'many': 1, 'bought': 1, 'product': 1, 'mush': 1, 'very': 1, 'never': 1, 'but': 2, 'direct': 1, 'they': 1, 'earlier': 1, 'exposure': 1, 'me': 1, 'car': 2, 'these': 1, 'of': 1, 'work': 1, 'well': 1, 'this': 1, 'protected': 1, 'keeps': 1, 'the': 2}


In [7]:
products[269]['word_count']

{'a': 1, 'favorite': 1, 'house': 1, 'in': 1, 'our': 1}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [8]:
print len(products)
products = products[ products['rating'] != 3 ]
print len(products)

183531
166752


Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [9]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
len(products) == len ( products[products['sentiment'] == -1]) + \
                len ( products[products['sentiment'] == 1])

True

In [51]:
products[ products['rating'] > 3].head(2)

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 3, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1


In [52]:
products[ products['rating'] < 3].head(2)

name,review,rating,word_count,sentiment
Nature's Lullabies SecondYear Sticker Calendar ...,I only purchased asecond-year calendar for ...,2.0,"{'and': 1, 'cute': 1,'selection': 1, 'just': ...",-1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2.0,"{'and': 3, 'rating': 1,'have': 1, 'just': 1, ...",-1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1). We can also see that all product reviews got a sentiment (+1 or -1) cause the sum of -1 and +1 sentiments add up to number of product reviews we have.

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [11]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [12]:
#Lets look at value of word_count column is 1st row/data point
train_data['word_count'].head(1)  #print train_data[0]['word_count']  

dtype: dict
Rows: 1
[{'and': 3, 'love': 1, 'it': 3, 'highly': 1, 'osocozy': 1, 'bags': 1, 'holder': 1, 'leak': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'early': 1, 'not': 2, 'now': 1, 'disappointed': 1, 'wipe': 1, 'keps': 1, 'wise': 1, 'i': 1, 'planet': 1, 'my': 2, 'came': 1}]

In [13]:
print train_data[0]['word_count'] 

{'and': 3, 'love': 1, 'it': 3, 'highly': 1, 'osocozy': 1, 'bags': 1, 'holder': 1, 'leak': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'early': 1, 'not': 2, 'now': 1, 'disappointed': 1, 'wipe': 1, 'keps': 1, 'wise': 1, 'i': 1, 'planet': 1, 'my': 2, 'came': 1}


In [53]:
print train_data['word_count'][0]

{'and': 3, 'love': 1, 'it': 3, 'highly': 1, 'osocozy': 1, 'bags': 1, 'holder': 1, 'leak': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'early': 1, 'not': 2, 'now': 1, 'disappointed': 1, 'wipe': 1, 'keps': 1, 'wise': 1, 'i': 1, 'planet': 1, 'my': 2, 'came': 1}


In [14]:
# if we could count num of unique words in word_count, that will be the the num of words and # of features

In [15]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 4.482241     | 0.960964          |

| 6         | 13       | 3.000000  | 4.871717     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [16]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 6
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 5.2699

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]       : 41.9847
word_count[placeid]            : 41.7354
word_count[labelbox]           : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid] : 40.0454
word_count[knobskeeping]       : 36.2091

Lowest Negative Coeffi

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [17]:
weights = sentiment_model.coefficients
# weights.column_names()
weights.head(10)

name,index,class,value,stderr
(intercept),None,1,1.30337080544,None
word_count,recommend,1,0.303815600015,None
word_count,moist,1,0.671556821415,None
word_count,osocozy,1,0.426326525702,None
word_count,keps,1,7.39633708721,None
word_count,leak,1,-0.24658014554,None
word_count,holder,1,-0.0300523581013,None
word_count,was,1,-0.0530004786379,None
word_count,now,1,0.0383787882079,None
word_count,wipe,1,0.165506649337,None


There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [18]:
num_positive_weights = len (weights[weights['value'] >= 0])
num_negative_weights = len (weights[weights['value'] < 0])

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights
print "Total: %s " % (num_positive_weights + num_negative_weights)

Number of positive weights: 68419 
Number of negative weights: 53294 
Total: 121713 


**Quiz question:** How many weights are >= 0?  68419

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [19]:
sample_test_data = test_data[10:13]  #samples/rows 10,11,12
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2, 'all': 1,'love': 1, 'purchased': ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1, 'would': 2,'almost': 1, 'decals' ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'all': 1, 'money': 1,'into': 1, 'back': 1, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [20]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [21]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**. Note that scores is just a dot product of weights and features as in regression!! - so predicted values, but what does do these scores or numbers mean??.

In [22]:
# margin classfier --> margin=score, 
# score >= 0 on one side of margin, score<0 on other side of margin or decision boundary
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores   
print scores.dtype

[6.734619727060203, -5.734130996760832, -14.668460404469426]
<bound method SArray.dtype of dtype: float
Rows: 3
[6.734619727060203, -5.734130996760832, -14.668460404469426]>


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions: 

Again there are many ways to skin the goat!!! Given scores is just a list of values, we can either use an in place lambda function of list comprehension to do this in one line, show off both ways below. Traditioanlly we would iterate through the list and pick up item/value one by one, inspect if its neg or pos and the store/print corresponding -1 or +1. In both cases a list of outcomes is produced.


In [23]:
predictions = scores.apply(lambda x : +1 if x > 0 else -1)  #lambda in place function
print predictions                                           #scores is SArray.dtype: float

[1, -1, -1]


In [24]:
#list comprehension  - optimized iterator through values in list, for each item do test
predictions = [+1 if x > 0 else -1 for x in scores]    
print predictions

[1, -1, -1]


In [25]:
def predict_class(model, data):
    scores = list(model.predict(data, output_type='margin'))
    score_class = [+1 if x > 0 else -1 for x in scores]
    return (score_class)  #if no enclose return, prints None!!

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [26]:
print "My class predictions:"
print predict_class(sentiment_model, sample_test_data)
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data)

My class predictions:
[1, -1, -1]
Class predictions according to GraphLab Create:
[1, -1, -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{Score})}.
$$ $$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**. 

Note that the probability values are for a prediction of positive sentiment (+1) given a specific value for featuere or input x and its weight/coefficient. so $$ P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) $$ 
We can easily find probability of a negative sentiment, just 1 - P(+1) !!

In [27]:
#double fApB = decision_value*A+B;
#if (fApB >= 0)
#    return Math.exp(-fApB)/(1.0+Math.exp(-fApB))
#else
#     return 1.0/(1+Math.exp(fApB))

import numpy as np
probabilities = 1.0 * 1/(1 + np.exp(-scores))
print probabilities

[  9.98812385e-01   3.22326818e-03   4.26155800e-07]


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [28]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

Class predictions according to GraphLab Create:
[0.9988123848377208, 0.003223268181798836, 4.2615579966515497e-07]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review? **third** cause it has lowest value to power of -7.

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [29]:
#scores = sentiment_model.predict(test_data, output_type='probability')
#scores.topk(20, reverse=True)    
#scores in not sframe - actually closest to 'SArray' object but has no attribute 'topk'
#so we add prob predictions as extra col in test_data, then sort by this column and print 1st 20 rows

# also don't do reverse=True  for topk, as it will then grab lowest prob values into top of list!!
test_data['prob_pred'] = sentiment_model.predict(test_data, output_type='probability')
topk = test_data.topk('prob_pred', 20)
topk.print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|   Munchkin Mozart Magic Cube  | My wife and I have been li... |  4.0   |
|  BABYBJORN Potty Chair - Red  | Our family is just startin... |  5.0   |
| Safety 1st Tot-Lok Four Lo... | I have a wooden desk that ... |  5.0   |
| Summer Infant Complete Nur... | This Nursery and Bath Care... |  4.0   |
| Leachco Snoogle Total Body... | I have had my Snoogle for ... |  5.0   |
| HALO SleepSack Micro-Fleec... | I love the Sleepsack weara... |  5.0   |
| Peg Perego Primo Viaggio C... | We have been using this se... |  5.0   |
|   Capri Stroller - Red Tech   | First let me say that I wa... |  4.0   |
| Wizard Convertible Car Sea... | My son was born big and re... |  5.0   |
| Britax Marathon Convertibl... | My son began using the Mar... |  5.0   |
| Britax Decathlon Conver

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [30]:
topk = test_data.topk('prob_pred', 20,reverse=True)
topk.print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Jolly Jumper Arctic Sneak ... | I am a "research-aholic" i... |  5.0   |
| Levana Safe N'See Digital ... | This is the first review I... |  1.0   |
| Snuza Portable Baby Moveme... | I would have given the pro... |  1.0   |
| Fisher-Price Ocean Wonders... | We have not had ANY luck w... |  2.0   |
| VTech Communications Safe ... | This is my second video mo... |  1.0   |
| Safety 1st High-Def Digita... | We bought this baby monito... |  1.0   |
| Chicco Cortina KeyFit 30 T... | My wife and I have used th... |  1.0   |
| Prince Lionheart Warmies W... | *****IMPORTANT UPDATE*****... |  1.0   |
| Valco Baby Tri-mode Twin S... | I give one star to the dim... |  1.0   |
| Adiri BPA Free Natural Nur... | I will try to write an obj... |  2.0   |
| Munchkin Nursery Projec

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [31]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    '''
    Many ways to do it 
    1.Get margin or scores and predict which side of decision bounday each sample falls
    by looking at the sign (+ve/-ve) of the score for each data point
    
      scores = model.predict(data, output_type='margin')
      predicted_labels = [+1 if x > 0 else -1 for x in scores]
    
    2.Get probabilities that a given sample is a +ve setiment given sample and weights
    and if prob >= 0.5, assign +ve +1 sentiment, -ve -1 otherwise
    
      probs = model.predict(data, output_type='probability')
      predicted_labels = [+1 if x >= 0.5 else -1 for x in probs]
    
    3. Easiest - get prediction in form of class labels +1 or -1
    default output of model.predict is class predictions
    
      predicted_labels =  model.predict(data, output_type='class')  
      
    '''
    predicted_labels =  model.predict(data) 
    
    # Compute the number of correctly classified examples
    # understand why this will always give num of data points regardless True/False
    # 1st to element-wise boolean comparision between the 2 arrays 
    # and then count how many comparisions returned True, i.e same class in both array !! 
    # but doesn't work - in fact (predicted_labels == true_labels) outputs 1s and 0s!!!!
    print "num_correct using len()", len ( (predicted_labels == true_labels) == 'True' )
    print (predicted_labels == true_labels)[0:10]
    print "num_correct using sum()",(predicted_labels == true_labels).sum() 
    #sum gives correct result as same as counting # of 1s, 1+0+1+0+0=2!!
    
    # np.equal(x1,x2) -- > Return (x1 == x2) element-wise
    # Parameters:	 x1, x2 : array_like : Input arrays of the same shape.
    # Returns:	  out : ndarray or bool :  Output array of bools, or a single bool if x1 and x2 are scalars.  
    
    print np.equal(predicted_labels, true_labels)[0:10]
    num_correct = sum ( np.equal(predicted_labels, true_labels) )
    print "num_correct", num_correct
    print "num_data", len(data)
    
    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = num_correct * 1.0/len(data)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [54]:
print "Accuracy = %0.2f" %get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

num_correct using len() 33336
[1, 1, 0, 1, 1, 1, 1, 0, 1, 1]
num_correct using sum() 30487
[ True  True False  True  True  True  True False  True  True]
num_correct 30487
num_data 33336
Accuracy = 0.91


**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76). **0.91**

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better? **No**

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [33]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [34]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [35]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [36]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [37]:
print train_data[0]['word_count']

{'and': 3, 'love': 1, 'it': 3, 'highly': 1, 'osocozy': 1, 'bags': 1, 'holder': 1, 'leak': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'early': 1, 'not': 2, 'now': 1, 'disappointed': 1, 'wipe': 1, 'keps': 1, 'wise': 1, 'i': 1, 'planet': 1, 'my': 2, 'came': 1}


Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [38]:
print train_data[0]['word_count_subset']

{'love': 1, 'disappointed': 1}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [39]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.333536     | 0.866815          |

| 6         | 7        | 0.390753     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.4067

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [40]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

num_correct using len() 33336
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
num_correct using sum() 28979
[ True  True  True  True  True  True  True False  True  True]
num_correct 28979
num_data 33336


0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [41]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,disappointed,1,-2.35509250061,0.0504149888557
word_count_subset,love,1,1.36543549368,0.0303546295109
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,product,1,-0.320555492996,0.0154311321362
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,great,1,0.94469126948,0.0209509926591
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,work,1,-0.621700012425,0.0230330597946


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [42]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [43]:
print len (simple_model.coefficients[ simple_model.coefficients['value'] > 0 ]) - 1

10


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [44]:
positive_significant_words = simple_model.coefficients [simple_model.coefficients['value'] > 0 ]['index']
print positive_significant_words

simple_positive = simple_model.coefficients.sort('value', ascending=False)
simple_positive = simple_positive [simple_positive['value'] > 0 ]['index']

setimental_positive = sentiment_model.coefficients.sort('value', ascending=False)
sentiment_positive = setimental_positive [setimental_positive['value'] > 0 ]['index']

print len(simple_positive)
print len(sentiment_positive)
simple_positive = simple_positive.to_numpy()
sentiment_positive = sentiment_positive.to_numpy()
print "These words have positive weights in both the models\n", np.intersect1d(simple_positive, sentiment_positive)

[None, 'love', 'little', 'loves', 'well', 'great', 'easy', 'able', 'perfect', 'old', 'car', ... ]
11
68419
These words have positive weights in both the models
[None 'able' 'car' 'easy' 'great' 'little' 'love' 'loves' 'old' 'perfect'
 'well']


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [45]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

num_correct using len() 133416
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
num_correct using sum() 130673
[ True  True  True  True  True  True  True  True  True  True]
num_correct 130673
num_data 133416


0.97944024704683097

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [46]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

num_correct using len() 133416
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
num_correct using sum() 115647
[ True  True  True  True  True  True  True  True  True  True]
num_correct 115647
num_data 133416


0.86681507465371466

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set? **sentiment_model**

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [47]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

num_correct using len() 33336
[1, 1, 0, 1, 1, 1, 1, 0, 1, 1]
num_correct using sum() 30487
[ True  True False  True  True  True  True False  True  True]
num_correct 30487
num_data 33336


0.91453683705303579

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [48]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

num_correct using len() 33336
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
num_correct using sum() 28979
[ True  True  True  True  True  True  True False  True  True]
num_correct 28979
num_data 33336


0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set? **sentiment_model**

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [49]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [50]:
num_positive  = (test_data['sentiment'] == +1).sum()
num_negative = (test_data['sentiment'] == -1).sum()
print num_positive
print num_negative
print round(num_positive/len(test_data), 2)

28095
5241
0.84


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)? Better 91% with train data, but with test data accuracy was 87%. Baseline accuracy from majority class classfier is 84%. So we can't say that sentiment_model is definitely better than baseline, as its accuracy is only 3% better.